In [9]:
import argparse
import socket
import selectors
import sys

sel = selectors.DefaultSelector()

counter = 0

class ClientData:
    def __init__(self, addr):
        self.addr = addr
        self.inb = b''
        self.outb = b''

    def __call__(self, addr, inb, outb):
        print("client data was called")


def connect(key, mask):
    # TODO: implement cli arguments
    print("socket is not connected")
    key.fileobj.connect(("131.94.128.43", 54634))
    print("socket is connected")


def serviceConnection(key, mask):
    sock = key.fileobj
    data = key.data

    #TODO: wait for the client to receive 2 "accio\r\n" messages
    if mask & selectors.EVENT_READ:
        # TODO: implement 10 second timeout
        try:
            recv_data = sock.recv(1024)
            if recv_data:
                data.outb += recv_data
            else:
                print('closing connection to', data.addr)
                sel.unregister(sock)
                sock.close()
        except KeyError:
            sys.stderr.write('ERROR: ')
    
    # TODO: sent packages in 10_000 bytes
    if mask & selectors.EVENT_WRITE:
        if data.outb:
            print('echoing', repr(data.outb), 'to', data.addr)
            # TODO: implement 10 second timeout
            try:
                sent = sock.send(data.outb)
                data.outb = data.outb[sent:]
            except KeyError:
                sys.stderr.write('ERROR: ')



# TODO: implement commandline "<HOSTNAME-OR-IP> <PORT> <FILENAME>"
parser = argparse.ArgumentParser(
                    prog = 'ProgramName',
                    description = 'What the program does',
                    epilog = 'Text at the bottom of help')


#TODO: exception handing for the hostname and port from the cli
# the file is always correct
# valid range for TCP port numbers (1-65535)
# ports are only 2 bytes 
try:
    pass
except KeyError:
    sys.stderr.write("ERROR: hostname or port is incorrect")


# TODO: change the client socket to blocking
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as clientSocket:
    clientSocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    print(clientSocket)

    # set socket to be non-blocking
    clientSocket.setblocking(False)

    # register the client socket with a clientData object
    data = ClientData("131.94.128.43")
    sel.register(clientSocket, selectors.EVENT_WRITE, data = data)

    # connect to the server
    clientSocket.connect(("131.94.128.43", 54634))

    # read the `read` and `write` events from the connection
    events = sel.select()
    while True:
        for key, mask in events:
            callback = key.data
            callback(key, mask)

<socket.socket fd=78, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 0)>


BlockingIOError: [Errno 115] Operation now in progress

In [35]:
import selectors
import socket

sel = selectors.DefaultSelector()

def accept(sock, mask):
    conn, addr = sock.accept()  # Should be ready
    print('accepted', conn, 'from', addr)
    conn.setblocking(False)
    sel.register(conn, selectors.EVENT_READ, read)

def read(conn, mask):
    data = conn.recv(1000)  # Should be ready
    if data:
        print('echoing', repr(data), 'to', conn)
        conn.send(data)  # Hope it won't block
    else:
        print('closing', conn)
        sel.unregister(conn)
        conn.close()

sock = socket.socket()
sock.bind(('localhost', 1234))
sock.listen(100)
sock.setblocking(False)
sel.register(sock, selectors.EVENT_READ, accept)

while True:
    events = sel.select()
    for key, mask in events:
        callback = key.data
        callback(key.fileobj, mask)

KeyboardInterrupt: 